In [ ]:
print("hi")

hi


In [2]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
file_path = '/content/drive/MyDrive/web3gle/Last Task/creditcard.csv'
data = pd.read_csv(file_path)
data.sample(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
216192,140378.0,1.959814,-1.224437,-1.982735,-1.840918,-0.429002,-1.114379,0.077315,-0.419829,-0.301721,...,-0.163651,-0.099826,-0.046085,-0.344112,0.056872,0.347358,-0.070817,-0.058660,141.35,0
101184,67751.0,-0.572872,1.012541,1.413513,-0.599945,0.462937,-0.555010,0.790269,0.041173,-0.773020,...,-0.164617,-0.597961,-0.122818,-0.048743,-0.212811,-0.068724,0.048267,0.110841,2.99,0
51781,45116.0,-1.672032,-0.770777,1.386309,1.460678,0.238928,0.679681,-0.752620,1.017070,-0.103070,...,0.339188,0.578752,0.077974,-0.267509,0.026028,-0.186265,-0.013722,-0.241595,96.85,0
120611,75890.0,1.179338,-0.174554,0.829131,0.057181,-0.957781,-0.673087,-0.444080,0.016743,0.191142,...,0.150140,0.395502,-0.016105,0.623964,0.168595,1.039134,-0.067559,0.004651,19.50,0
203966,135052.0,1.839303,-0.658990,-1.318029,0.271534,1.650956,4.230737,-1.334916,1.159807,1.013593,...,0.074062,0.180545,0.282958,0.660041,-0.314983,-0.738293,0.090650,-0.019662,45.00,0


In [ ]:
# Data Preprocessing
# Check for missing values
print("Missing Values in Each Column:\n", data.isnull().sum())

# Basic statistics
print("\nBasic Statistics:\n", data.describe())

# Checking class distribution
print("\nClass Distribution:\n", data['Class'].value_counts(normalize=True))

# Feature and target variables
X = data.drop('Class', axis=1)
y = data['Class']

# Exploratory Data Analysis (EDA)
# Distribution of the 'Amount' feature
plt.figure(figsize=(10,6))
sns.histplot(data['Amount'], bins=50, kde=True)
plt.title('Distribution of Transaction Amount')
plt.xlabel('Amount ($)')
plt.ylabel('Frequency')
plt.show()

# Distribution of time variable
plt.figure(figsize=(10,6))
sns.histplot(data['Time'], bins=50, kde=True)
plt.title('Distribution of Transaction Time')
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency')
plt.show()

# Correlation heatmap
plt.figure(figsize=(20,15))
corr_matrix = data.corr()
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

# Checking for correlations with the target variable
corr_with_target = corr_matrix['Class'].sort_values(ascending=False)
print("\nCorrelation of Features with the Target Variable:\n", corr_with_target)

# Boxplot for feature 'V1' to 'V28' and their relation to the target variable
plt.figure(figsize=(20,40))
for i, col in enumerate(data.columns[1:29]):
    plt.subplot(10, 3, i+1)
    sns.boxplot(x='Class', y=col, data=data)
    plt.title(f'Boxplot of {col} by Class')
plt.tight_layout()
plt.show()

# Feature Scaling (Normalize Amount and Time)
scaler = StandardScaler()
data[['Amount', 'Time']] = scaler.fit_transform(data[['Amount', 'Time']])

# Feature and target variables after scaling
X = data.drop('Class', axis=1)
y = data['Class']

Missing Values in Each Column:
 Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

Basic Statistics:
                 Time            V1            V2            V3            V4  \
count  284807.000000  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05   
mean    94813.859575  1.168375e-15  3.416908e-16 -1.379537e-15  2.074095e-15   
std     47488.145955  1.958696e+00  1.651309e+00  1.516255e+00  1.415869e+00   
min         0.000000 -5.640751e+01 -7.271573e+01 -4.832559e+01 -5.683171e+00   
25%     54201.500000 -9.203734e-01 -5.985499e-01 -8.903648e-01 -8.486401e-01   
50%     84692.000000  1.810880e-02  6.548556e-02  1.798463e-01 -1.984653e-02   
75%

In [ ]:
# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Classification algorithms
classifiers = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'Support Vector Classifier': SVC(probability=True),
    'K-Nearest Neighbors': KNeighborsClassifier()
}


In [ ]:
# Define a function for evaluating performance
def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"Classification Report for {model.__class__.__name__}:\n", classification_report(y_test, y_pred))
    print(f"Confusion Matrix for {model.__class__.__name__}:\n", confusion_matrix(y_test, y_pred))
    print(f"AUC-ROC Score for {model.__class__.__name__}: {roc_auc_score(y_test, model.predict_proba(X_test)[:,1])}\n")

# Baseline performance evaluation
print("Baseline Performance:\n")
for name, clf in classifiers.items():
    print(f"Evaluating {name}...")
    evaluate_model(clf, X_train, y_train, X_test, y_test)

# Handling Class Imbalance
# Resampling Methods
# SMOTE
smote = SMOTE(random_state=42)
X_resampled_smote, y_resampled_smote = smote.fit_resample(X_train, y_train)
print("SMOTE Resampling Performance:\n")
for name, clf in classifiers.items():
    print(f"Evaluating {name} with SMOTE...")
    evaluate_model(clf, X_resampled_smote, y_resampled_smote, X_test, y_test)

# Random Under Sampling
undersampler = RandomUnderSampler(random_state=42)
X_resampled_undersample, y_resampled_undersample = undersampler.fit_resample(X_train, y_train)
print("Random Under Sampling Performance:\n")
for name, clf in classifiers.items():
    print(f"Evaluating {name} with Random Under Sampling...")
    evaluate_model(clf, X_resampled_undersample, y_resampled_undersample, X_test, y_test)

# Combined SMOTE + Edited Nearest Neighbors (SMOTEENN)
smote_enn = SMOTEENN(random_state=42)
X_resampled_smoteenn, y_resampled_smoteenn = smote_enn.fit_resample(X_train, y_train)
print("SMOTEENN Resampling Performance:\n")
for name, clf in classifiers.items():
    print(f"Evaluating {name} with SMOTEENN...")
    evaluate_model(clf, X_resampled_smoteenn, y_resampled_smoteenn, X_test, y_test)



In [ ]:
# Algorithmic Methods
# Balanced Random Forest
balanced_rf = BalancedRandomForestClassifier(random_state=42)
print("Balanced Random Forest Performance:\n")
evaluate_model(balanced_rf, X_train, y_train, X_test, y_test)

# Compute class weights for Logistic Regression
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))
print(f"Class Weights: {class_weight_dict}")

# Logistic Regression with Class Weights
logistic_weighted = LogisticRegression(class_weight=class_weight_dict, random_state=42)
print("Logistic Regression with Class Weights Performance:\n")
evaluate_model(logistic_weighted, X_train, y_train, X_test, y_test)

